# CFT – Heart Rate (Variability) Analysis

## Setup and Helper Functions

In [1]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

pg.options['round'] = 4

palette

[(0.0, 0.2196078431372549, 0.396078431372549),
 (0.596078431372549, 0.6431372549019608, 0.6823529411764706),
 (0.788235294117647, 0.5764705882352941, 0.07450980392156863),
 (0.0, 0.6941176470588235, 0.9215686274509803),
 (0.0, 0.6078431372549019, 0.4666666666666667),
 (0.5529411764705883, 0.0784313725490196, 0.1607843137254902)]

## Import Data

In [3]:
base_path = Path("../../../")
data_path = base_path.joinpath("data/for_analysis")

results_path = base_path.joinpath("results")
plot_path = results_path.joinpath("plots")
stats_path = results_path.joinpath("statistics")

bp.utils.file_handling.mkdirs([results_path, plot_path, stats_path])

In [4]:
hr_measures = bp.io.load_wide_format_csv(
    data_path.joinpath("cft_measures_merged.csv")
)

cft_parameter = bp.io.load_wide_format_csv(
    data_path.joinpath("cft_parameter.csv"),
    index_cols=["condition", "subject", "phase"]
)

hr_ensemble = bp.io.load_pandas_dict_excel(
    data_path.joinpath("cft_hr_ensemble.xlsx")
)

In [5]:
hue_order = ["Control", "CFT"]

## Response to the MIST

### Descriptive Analysis

To show the general efficacy of the MIST we compute the mean and maximum heart rate increase for each subject and MIST phase, respectively:

In [6]:
hr_increase = pd.concat({"HR": pd.concat({key: hr_df.agg(["mean", "max"]) for key, hr_df in hr_ensemble.items()}).T.stack(level=0)}, axis=1)

hr_increase.index = hr_increase.index.set_names(["subject", "phase"])
hr_increase.head()

HR          
                     max      mean
subject phase                     
Vp01    MIST1  12.470026 -6.787072
        MIST2  16.322364 -9.014519
        MIST3  22.173485 -4.695198
Vp02    MIST1  23.986697 -2.053367
        MIST2  27.679596  4.250668

Afterwards, we compute mean and standard deviation over all subject:

In [7]:
hr_increase_agg = hr_increase.groupby("phase").agg(["mean", "std"]).T

hr_increase_agg.to_csv(results_path.joinpath("hr_increase_mist.csv"))
hr_increase_agg

phase             MIST1      MIST2      MIST3
HR max  mean  25.997396  32.895060  39.020036
        std   10.503582  15.385118  17.134969
   mean mean  -1.981908   3.852439  10.320102
        std    5.375256  10.541039  12.849598

### Statistical Analysis

**Population**: Control and CFT group separately

**Analysis** (per measure):
1. *Increase BL-AT*: Check whether Arithmetic Tasks of the MIST causes significant HR(V) responses.
    * Procedure: Paired t-tests between subphases *BL* and *AT* for each MIST phase
    * Expected Result: Significant HR(V) responses for each MIST phase
2. *Increase over MIST*: Check whether the HR response increases over tiem (i.e., over MIST phases). 
    * Procedure: Repeated-measures ANOVA for each subphase
    * *(only for HR)*: Post-hoc test to check which pairs of MIST phases show significant differences
    * Expected Result: Significant main effect *MIST Phase* for each subphase


**Findings**: 
* Each MIST Phase induces stress, indicated by significant differences between $BL_{Loc}$ and $AT$ in each individual MIST Phase
* Stress levels increase over time, indicated by significant main effect *MIST Phase* and significant differences between MIST phases, especially between *MIST1* and *MIST3*

#### 1. Increase BL-AT

##### Prepare Data

In [8]:
measures = ["HR_Norm", "HRV_RMSSD","HRV_pNN20"]
subphases = ["BL", "AT"]

# Select data
hr_analysis = hr_measures.loc[pd.IndexSlice[:, :, :, subphases, measures]].copy()
hr_analysis.head()

data
condition subject phase subphase name              
CFT       Vp01    MIST1 BL       HR_Norm  -4.845864
                  MIST2 BL       HR_Norm  -5.617980
                  MIST3 BL       HR_Norm -10.087795
          Vp02    MIST1 BL       HR_Norm  -2.322634
                  MIST2 BL       HR_Norm   0.950141

##### Statistics

In [9]:
steps = [
    ("prep", "normality"),
    ("test", "pairwise_ttests")
]
params = {
    "dv": "data",
    "within": "subphase",
    "subject": "subject",
    "groupby": ["condition", "name", "phase"],
    "multicomp": {"levels": "phase"}
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(hr_analysis)

stats.export_statistics(stats_path.joinpath("stats_hr_response_mist_increase_bl_at.xlsx"))
stats.display_results(prep=False)

<font size="3"><b> Overview </b></font>

,dv,within,subject,groupby,multicomp
parameter,data,subphase,subject,"['condition', 'name', 'phase']",{'levels': 'phase'}


,prep,test
parameter,normality,pairwise_ttests


<font size="3"><b> Statistical Tests </b></font>

**Pairwise t-Tests**

Contrast   A   B  Paired  Parametric       T  \
condition name      phase                                                   
CFT       HR_Norm   MIST1 0  subphase  AT  BL    True        True  0.8034   
                    MIST2 0  subphase  AT  BL    True        True  4.0857   
                    MIST3 0  subphase  AT  BL    True        True  6.4911   
Control   HR_Norm   MIST1 0  subphase  AT  BL    True        True  3.3113   
                    MIST2 0  subphase  AT  BL    True        True  4.1992   
                    MIST3 0  subphase  AT  BL    True        True  3.8499   
CFT       HRV_RMSSD MIST1 0  subphase  AT  BL    True        True  2.0577   
                    MIST2 0  subphase  AT  BL    True        True -1.4039   
                    MIST3 0  subphase  AT  BL    True        True -3.9109   
Control   HRV_RMSSD MIST1 0  subphase  AT  BL    True        True -1.2899   
                    MIST2 0  subphase  AT  BL    True        True -3.1026   
                    MIST3 0  subphase  AT  BL    True        True -1.9357   
CFT       HRV_pNN20 MIST1 0  subphase  AT  BL    True        True  2.5148   
                    MIST2 0  subphase  AT  BL    True        True -1.1971   
                    MIST3 0  subphase  AT  BL    True        True -3.2867   
Control   HRV_pNN20 MIST1 0  subphase  AT  BL    True        True -1.0078   
                    MIST2 0  subphase  AT  BL    True        True -3.0325   
                    MIST3 0  subphase  AT  BL    True        True -2.1916   

                              dof alternative   p-unc     BF10  hedges  p-corr  
condition name      phase                                                       
CFT       HR_Norm   MIST1 0  11.0   two-sided  0.4387    0.378  0.2507  1.0000  
                    MIST2 0  11.0   two-sided  0.0018   24.382  1.3587  0.0054  
                    MIST3 0  11.0   two-sided  0.0000  579.528  1.8857  0.0000  
Control   HR_Norm   MIST1 0  12.0   two-sided  0.0062    8.564  0.6553  0.0186  
                    MIST2 0  12.0   two-sided  0.0012   33.143  0.8936  0.0036  
                    MIST3 0  12.0   two-sided  0.0023   19.512  0.7616  0.0069  
CFT       HRV_RMSSD MIST1 0  11.0   two-sided  0.0641    1.373  0.4234  0.1923  
                    MIST2 0  11.0   two-sided  0.1879    0.634 -0.2151  0.5637  
                    MIST3 0  11.0   two-sided  0.0024   18.985 -0.4607  0.0072  
Control   HRV_RMSSD MIST1 0  12.0   two-sided  0.2214    0.552 -0.2024  0.6642  
                    MIST2 0  12.0   two-sided  0.0091    6.231 -0.5338  0.0273  
                    MIST3 0  12.0   two-sided  0.0768    1.166 -0.3515  0.2304  
CFT       HRV_pNN20 MIST1 0  11.0   two-sided  0.0287    2.546  0.4266  0.0861  
                    MIST2 0  11.0   two-sided  0.2564    0.518 -0.2437  0.7692  
                    MIST3 0  11.0   two-sided  0.0072    7.693 -0.7067  0.0216  
Control   HRV_pNN20 MIST1 0  12.0   two-sided  0.3335    0.427 -0.1198  1.0000  
                    MIST2 0  12.0   two-sided  0.0104    5.603 -0.5160  0.0312  
                    MIST3 0  12.0   two-sided  0.0489    1.643 -0.2842  0.1467

#### 2. Increase over MIST

##### Prepare Data

In [10]:
measures = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"]
conditions = ["Control"]

# Select data
hr_analysis = hr_measures.loc[pd.IndexSlice[:, :, :, :, measures]].copy()
hr_analysis.head()

data
condition subject phase subphase name              
CFT       Vp01    MIST1 AT       HR_Norm  -4.866102
                        BL       HR_Norm  -4.845864
                        FB       HR_Norm  -5.795996
                        RP_CFI   HR_Norm -12.341485
                  MIST2 AT       HR_Norm  -7.483283

##### Statistics

In [11]:
steps = [
    ("prep", "normality"),
    ("prep", "equal_var"),
    ("test", "rm_anova"),
    ("posthoc", "pairwise_ttests")
]
params = {
    "dv": "data",
    "within": "phase",
    "subject": "subject",
    "groupby": ["condition", "name", "subphase"],
    "multicomp": {"levels": False}
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(hr_analysis)

stats.export_statistics(stats_path.joinpath("stats_hr_response_mist_increase_mist_phases.xlsx"))
stats.display_results(prep=False, sig_only="posthoc")

<font size="3"><b> Overview </b></font>

,dv,within,subject,groupby,multicomp
parameter,data,phase,subject,"['condition', 'name', 'subphase']",{'levels': False}


,prep,prep,test,posthoc
parameter,normality,equal_var,rm_anova,pairwise_ttests


<font size="3"><b> Statistical Tests </b></font>

**Repeated-measurement ANOVA**

Source  ddof1  ddof2        F   p-unc     np2  \
condition name      subphase                                                   
CFT       HR_Norm   AT       0  phase      2     22  32.3817  0.0000  0.7464   
                    BL       0  phase      2     22   0.2438  0.7857  0.0217   
                    FB       0  phase      2     22  10.3062  0.0007  0.4837   
                    RP_CFI   0  phase      2     22   7.0493  0.0043  0.3906   
Control   HR_Norm   AT       0  phase      2     24   9.1600  0.0011  0.4329   
                    BL       0  phase      2     24   7.9027  0.0023  0.3971   
                    FB       0  phase      2     24   9.2494  0.0011  0.4353   
                    RP_CFI   0  phase      2     24   9.5197  0.0009  0.4424   
CFT       HRV_RMSSD AT       0  phase      2     22  15.0627  0.0001  0.5779   
                    BL       0  phase      2     22   1.5927  0.2259  0.1265   
                    FB       0  phase      2     22   0.7991  0.4624  0.0677   
                    RP_CFI   0  phase      2     22   2.0506  0.1525  0.1571   
Control   HRV_RMSSD AT       0  phase      2     24   8.3585  0.0018  0.4106   
                    BL       0  phase      2     24   2.4660  0.1062  0.1705   
                    FB       0  phase      2     24   7.3983  0.0031  0.3814   
                    RP_CFI   0  phase      2     24   0.5769  0.5692  0.0459   
CFT       HRV_pNN50 AT       0  phase      2     22  11.4284  0.0004  0.5096   
                    BL       0  phase      2     22   2.4852  0.1064  0.1843   
                    FB       0  phase      2     22   0.0335  0.9671  0.0030   
                    RP_CFI   0  phase      2     22   0.4407  0.6491  0.0385   
Control   HRV_pNN50 AT       0  phase      2     24   8.1893  0.0019  0.4056   
                    BL       0  phase      2     24   1.8156  0.1844  0.1314   
                    FB       0  phase      2     24   1.8550  0.1782  0.1339   
                    RP_CFI   0  phase      2     24   0.5387  0.5904  0.0430   

                                   eps  p-GG-corr sphericity  W-spher  p-spher  
condition name      subphase                                                    
CFT       HR_Norm   AT       0  0.7682        NaN        NaN      NaN      NaN  
                    BL       0  0.9025        NaN        NaN      NaN      NaN  
                    FB       0  0.6585     0.0038      False   0.4813   0.0258  
                    RP_CFI   0  0.6040     0.0158      False   0.3444   0.0048  
Control   HR_Norm   AT       0  0.9646        NaN        NaN      NaN      NaN  
                    BL       0  0.9637        NaN        NaN      NaN      NaN  
                    FB       0  0.7786        NaN        NaN      NaN      NaN  
                    RP_CFI   0  0.8299        NaN        NaN      NaN      NaN  
CFT       HRV_RMSSD AT       0  0.7508        NaN        NaN      NaN      NaN  
                    BL       0  0.9389        NaN        NaN      NaN      NaN  
                    FB       0  0.6852     0.4236      False   0.5406   0.0462  
                    RP_CFI   0  0.7022        NaN        NaN      NaN      NaN  
Control   HRV_RMSSD AT       0  0.9129        NaN        NaN      NaN      NaN  
                    BL       0  0.9172        NaN        NaN      NaN      NaN  
                    FB       0  0.8670        NaN        NaN      NaN      NaN  
                    RP_CFI   0  0.9466        NaN        NaN      NaN      NaN  
CFT       HRV_pNN50 AT       0  0.7345        NaN        NaN      NaN      NaN  
                    BL       0  0.9897        NaN        NaN      NaN      NaN  
                    FB       0  0.5872     0.8913      False   0.2970   0.0023  
                    RP_CFI   0  0.6636     0.5723      False   0.4930   0.0291  
Control   HRV_pNN50 AT       0  0.8491        NaN        NaN      NaN      NaN  
                    BL       0  0.9056        NaN        NaN      NaN      NaN  
      

<font size="3"><b> Post-Hoc Analysis </b></font>

**Pairwise t-Tests**

Contrast      A      B  Paired  Parametric  \
condition name      subphase                                                
CFT       HR_Norm   AT       0    phase  MIST1  MIST2    True        True   
                             1    phase  MIST1  MIST3    True        True   
                             2    phase  MIST2  MIST3    True        True   
                    FB       1    phase  MIST1  MIST3    True        True   
                    RP_CFI   2    phase  MIST2  MIST3    True        True   
Control   HR_Norm   AT       0    phase  MIST1  MIST2    True        True   
                             1    phase  MIST1  MIST3    True        True   
                    BL       1    phase  MIST1  MIST3    True        True   
                    FB       1    phase  MIST1  MIST3    True        True   
                    RP_CFI   1    phase  MIST1  MIST3    True        True   
                             2    phase  MIST2  MIST3    True        True   
CFT       HRV_RMSSD AT       1    phase  MIST1  MIST3    True        True   
                             2    phase  MIST2  MIST3    True        True   
Control   HRV_RMSSD AT       1    phase  MIST1  MIST3    True        True   
                    FB       1    phase  MIST1  MIST3    True        True   
                             2    phase  MIST2  MIST3    True        True   
CFT       HRV_pNN50 AT       1    phase  MIST1  MIST3    True        True   
                             2    phase  MIST2  MIST3    True        True   
Control   HRV_pNN50 AT       0    phase  MIST1  MIST2    True        True   
                             1    phase  MIST1  MIST3    True        True   

                                     T   dof alternative   p-unc     BF10  \
condition name      subphase                                                
CFT       HR_Norm   AT       0 -4.3573  11.0   two-sided  0.0011   35.828   
                             1 -6.4603  11.0   two-sided  0.0000  558.574   
                             2 -5.0406  11.0   two-sided  0.0004   91.793   
                    FB       1 -3.9062  11.0   two-sided  0.0025   18.859   
                    RP_CFI   2 -2.8576  11.0   two-sided  0.0156    4.138   
Control   HR_Norm   AT       0 -2.8317  12.0   two-sided  0.0151    4.141   
                             1 -4.1871  12.0   two-sided  0.0013   32.546   
                    BL       1 -4.0193  12.0   two-sided  0.0017   25.248   
                    FB       1 -3.5281  12.0   two-sided  0.0042   11.931   
                    RP_CFI   1 -3.4609  12.0   two-sided  0.0047   10.765   
                             2 -3.7258  12.0   two-sided  0.0029   16.143   
CFT       HRV_RMSSD AT       1  4.3969  11.0   two-sided  0.0011   37.875   
                             2  4.3767  11.0   two-sided  0.0011   36.817   
Control   HRV_RMSSD AT       1  3.5877  12.0   two-sided  0.0037   13.071   
                    FB       1  3.1000  12.0   two-sided  0.0092    6.206   
                             2  3.2918  12.0   two-sided  0.0064    8.312   
CFT       HRV_pNN50 AT       1  3.9140  11.0   two-sided  0.0024    19.07   
                             2  4.0685  11.0   two-sided  0.0019   23.794   
Control   HRV_pNN50 AT       0  2.8786  12.0   two-sided  0.0139    4.443   
                             1  3.3882  12.0   two-sided  0.0054    9.632   

                                hedges  p-corr  
condition name      subphase                    
CFT       HR_Norm   AT       0 -0.9301  0.0033  
                             1 -1.6057  0.0000  
                             2 -0.8188  0.0012  
                    FB       1 -1.1822  0.0075  
                    RP_CFI   2 -0.7946  0.0468  
Control   HR_Norm   AT       0 -0.8920  0.0453  
                             1 -1.3317  0.0039  
                    BL       1 -1.0276  0.0051  
                    FB       1 -1.0123  0.0126  
                    RP_CFI   1 -1.2191  0.0141  
                             2 -0.7925  0.0087  
CFT 

## Response to the CFT

### Descriptive Analysis

In [12]:
features = ["onset_latency", "peak_brady_latency", "peak_brady_percent", "mean_brady_percent"]

In [13]:
cft_analysis = cft_parameter[features]
cft_analysis_agg = cft_analysis.groupby("phase").agg(
    {
        "onset_latency": ["mean", "std"], 
        "peak_brady_latency": ["mean", "std"], 
        "peak_brady_percent": ["min", "mean", "std"], 
        "mean_brady_percent": ["min", "mean", "std"]
    }
).T
cft_analysis_agg

phase                        MIST1      MIST2      MIST3
onset_latency      mean   6.962786   3.897429  10.657214
                   std    8.207584   4.317788   7.386746
peak_brady_latency mean  49.548143  49.731714  49.824857
                   std   30.455020  32.436207  31.181355
peak_brady_percent min  -26.617372 -23.891467 -20.651473
                   mean -18.156020 -15.187533 -11.757308
                   std    5.153664   5.248955   3.789015
mean_brady_percent min  -15.320881 -11.770874  -7.061865
                   mean  -5.614254  -3.416494  -1.252645
                   std    5.871743   4.484611   4.146159

In [14]:
cft_time_bl_glo = hr_measures.loc[pd.IndexSlice["CFT", :, :, "RP_CFI", "Time_BL_Glo_HR"]]

cft_time_bl_glo.groupby(["phase"]).agg(["mean", "std"]).T * 100

phase            MIST1        MIST2        MIST3
data mean   938.787775  1019.963329  2012.113449
     std   1526.760868   897.632980  1750.587419

### Statistical Analysis

**Population**: CFT group  
**Analysis** (per measure):
1. *Decrease BL-RP_CFI*: Check whether Cold Face Intervention causes significant HR(V) responses.
    * Procedure: Paired t-tests between subphases *BL* and *RP_CFI* for each MIST phase
    * Expected Result: Significant HR(V) responses during each MIST phase


**Findings**: 
* Each MIST Phase induces stress, indicated by significant differences between $BL_{Loc}$ and $AT$ in each individual MIST Phase
* Stress levels increase over time, indicated by significant main effect *MIST Phase* and significant differences between MIST phases, especially between *MIST1* and *MIST3*

#### 1. Decrease *BL-RP_CFI*

##### Prepare Data

In [15]:
measures = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"]
subphases = ["BL", "RP_CFI"]
conditions = ["CFT"]

# Select data
hr_analysis = hr_measures.loc[pd.IndexSlice[conditions, :, :, subphases, measures]].copy()
hr_analysis.head()

data
condition subject phase subphase name              
CFT       Vp01    MIST1 BL       HR_Norm  -4.845864
                  MIST2 BL       HR_Norm  -5.617980
                  MIST3 BL       HR_Norm -10.087795
          Vp02    MIST1 BL       HR_Norm  -2.322634
                  MIST2 BL       HR_Norm   0.950141

##### Statistics

In [16]:
steps = [
    ("prep", "normality"),
    ("test", "pairwise_ttests")
]
params = {
    "dv": "data",
    "within": "subphase",
    "subject": "subject",
    "groupby": ["name", "phase"],
    "multicomp": {"levels": "phase"}
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(hr_analysis)

stats.export_statistics(stats_path.joinpath("stats_hr_response_cft_increase_bl_cfi.xlsx"))
stats.display_results(prep=False)

<font size="3"><b> Overview </b></font>

,dv,within,subject,groupby,multicomp
parameter,data,subphase,subject,"['name', 'phase']",{'levels': 'phase'}


,prep,test
parameter,normality,pairwise_ttests


<font size="3"><b> Statistical Tests </b></font>

**Pairwise t-Tests**

Contrast   A       B  Paired  Parametric       T   dof  \
name      phase                                                             
HR_Norm   MIST1 0  subphase  BL  RP_CFI    True        True  3.8449  11.0   
          MIST2 0  subphase  BL  RP_CFI    True        True  2.9717  11.0   
          MIST3 0  subphase  BL  RP_CFI    True        True  0.6733  11.0   
HRV_RMSSD MIST1 0  subphase  BL  RP_CFI    True        True -2.6815  11.0   
          MIST2 0  subphase  BL  RP_CFI    True        True -1.4346  11.0   
          MIST3 0  subphase  BL  RP_CFI    True        True -1.2136  11.0   
HRV_pNN50 MIST1 0  subphase  BL  RP_CFI    True        True -3.1493  11.0   
          MIST2 0  subphase  BL  RP_CFI    True        True -1.2922  11.0   
          MIST3 0  subphase  BL  RP_CFI    True        True -3.1291  11.0   

                  alternative   p-unc    BF10  hedges  p-corr  
name      phase                                                
HR_Norm   MIST1 0   two-sided  0.0027  17.267  1.2551  0.0081  
          MIST2 0   two-sided  0.0127   4.875  0.9477  0.0381  
          MIST3 0   two-sided  0.5147   0.349  0.1675  1.0000  
HRV_RMSSD MIST1 0   two-sided  0.0214   3.219 -0.4877  0.0642  
          MIST2 0   two-sided  0.1792   0.655 -0.1761  0.5376  
          MIST3 0   two-sided  0.2503   0.526 -0.1433  0.7509  
HRV_pNN50 MIST1 0   two-sided  0.0093   6.302 -0.5429  0.0279  
          MIST2 0   two-sided  0.2228   0.567 -0.1737  0.6684  
          MIST3 0   two-sided  0.0096    6.12 -0.4148  0.0288

## Effect of CFT on Acute Stress Response

### Statistical Analysis

**Population**: Control vs. CFT group  

**Analysis** (per measure):
1. *Interaction Condition x MIST Phase*: Check whether Cold Face Intervention has an effect on the HR(V) responses to the MIST.
    * Procedure:
        * Mixed-ANOVA to determine interaction effect between *Condition* and *MIST Phase*
        * In case of significant interaction effect: Post-hoc test to assess during which MIST Phase the conditions showed HR(V) response differences
    * Expected Result: Significant interaction effect between *Condition* and *MIST Phase*


**Findings**: 
* Each MIST Phase induces stress, indicated by significant differences between $BL_{Loc}$ and $AT$ in each individual MIST Phase
* Stress levels increase over time, indicated by significant main effect *MIST Phase* and significant differences between MIST phases, especially between *MIST1* and *MIST3*

##### Prepare Data

In [17]:
measures = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50", "Time_BL_Glo_HR", "Time_BL_Glo_HRV_RMSSD", "Time_BL_Glo_HRV_pNN50"]

# Select data
hr_analysis = hr_measures.loc[pd.IndexSlice[:, :, :, :, measures]].copy()
hr_analysis.head()

data
condition subject phase subphase name              
CFT       Vp01    MIST1 AT       HR_Norm  -4.866102
                        BL       HR_Norm  -4.845864
                        FB       HR_Norm  -5.795996
                        RP_CFI   HR_Norm -12.341485
                  MIST2 AT       HR_Norm  -7.483283

##### Statistics

In [18]:
steps = [
    ("prep", "normality"),
    ("prep", "equal_var"),
    ("test", "mixed_anova"),
    ("posthoc", "pairwise_ttests")
]
params = {
    "dv": "data",
    "within": "phase",
    "between": "condition",
    "subject": "subject",
    "groupby": ["name", "subphase"],
    "padjust": "bonf"
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(hr_analysis)

stats.export_statistics(stats_path.joinpath("stats_effect_cft_interaction.xlsx"))
stats.display_results(prep=False, sig_only=["test", "posthoc"])

<font size="3"><b> Overview </b></font>

,dv,within,between,subject,groupby,padjust
parameter,data,phase,condition,subject,"['name', 'subphase']",bonf


,prep,prep,test,posthoc
parameter,normality,equal_var,mixed_anova,pairwise_ttests


<font size="3"><b> Statistical Tests </b></font>

**Mixed ANOVA**

Source          SS  DF1  DF2  \
name                  subphase                                        
HR_Norm               AT       0    condition   1701.0035    1   23   
                               1        phase   3073.1330    2   46   
                      BL       0    condition   1366.2326    1   23   
                               1        phase    363.8055    2   46   
                               2  Interaction    370.3966    2   46   
                      FB       0    condition   1010.6252    1   23   
                               1        phase   1102.3965    2   46   
                      RP_CFI   0    condition   2956.6659    1   23   
                               1        phase    854.6626    2   46   
HRV_RMSSD             AT       1        phase   1761.2888    2   46   
                      FB       1        phase    455.4796    2   46   
HRV_pNN50             AT       1        phase   1276.7477    2   46   
Time_BL_Glo_HR        AT       1        phase  15370.3950    2   46   
                      BL       0    condition  12251.2716    1   23   
                               1        phase   3819.0293    2   46   
                               2  Interaction   2788.8848    2   46   
                      FB       0    condition   5393.3441    1   23   
                               1        phase  10227.1932    2   46   
                      RP_CFI   0    condition  32257.8113    1   23   
                               1        phase   7808.4064    2   46   
                               2  Interaction   1971.8838    2   46   
Time_BL_Glo_HRV_RMSSD AT       0    condition   6763.9764    1   23   
                               1        phase   7048.7939    2   46   
                      FB       0    condition   4983.1369    1   23   
                               1        phase   1665.7547    2   46   
                      RP_CFI   0    condition  14365.2037    1   23   
Time_BL_Glo_HRV_pNN50 AT       1        phase   8828.7597    2   46   
                      RP_CFI   0    condition  11324.8697    1   23   

                                          MS        F   p-unc     np2     eps  \
name                  subphase                                                  
HR_Norm               AT       0   1701.0035   6.7937  0.0158  0.2280     NaN   
                               1   1536.5665  22.9113  0.0000  0.4990  0.9704   
                      BL       0   1366.2326  12.4245  0.0018  0.3507     NaN   
                               1    181.9028   6.1499  0.0043  0.2110  0.9630   
                               2    185.1983   6.2613  0.0039  0.2140     NaN   
                      FB       0   1010.6252   6.6700  0.0166  0.2248     NaN   
                               1    551.1982  18.5448  0.0000  0.4464  0.7525   
                      RP_CFI   0   2956.6659  34.0692  0.0000  0.5970     NaN   
                               1    427.3313  15.3479  0.0000  0.4002  0.7766   
HRV_RMSSD             AT       1    880.6444  19.9223  0.0000  0.4641  0.8892   
                      FB       1    227.7398   6.1313  0.0044  0.2105  0.8335   
HRV_pNN50             AT       1    638.3739  18.0332  0.0000  0.4395  0.8396   
Time_BL_Glo_HR        AT       1   7685.1975  34.8393  0.0000  0.6023  0.9403   
                      BL       0  12251.2716  13.8897  0.0011  0.3765     NaN   
                               1   1909.5146   4.7562  0.0133  0.1714  0.9992   
                               2   1394.4424   3.4733  0.0394  0.1312     NaN   
                      FB       0   5393.3441   5.5186  0.0278  0.1935     NaN   
                               1   5113.5966  22.5305  0.0000  0.4948  0.7393   
                      RP_CFI   0  32257.8113  76.3949  0.0000  0.7686     NaN   
                               1   3904.2032  13.3388  0.0000  0.3671  0.9064   
                               2    985.9419   3.3685  0.0431  0.1277     NaN   
Time_BL_Glo_HRV_RMSSD AT       0   6763.9764 

<font size="3"><b> Post-Hoc Analysis </b></font>

**Pairwise t-Tests**

Contrast  phase      A        B  \
name                  subphase                                               
HR_Norm               AT       0              phase      -  MIST1    MIST2   
                               1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
                               4  phase * condition  MIST1    CFT  Control   
                      BL       1              phase      -  MIST1    MIST3   
                               5  phase * condition  MIST2    CFT  Control   
                               6  phase * condition  MIST3    CFT  Control   
                      FB       0              phase      -  MIST1    MIST2   
                               1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
                               4  phase * condition  MIST1    CFT  Control   
                      RP_CFI   1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
                               4  phase * condition  MIST1    CFT  Control   
                               5  phase * condition  MIST2    CFT  Control   
                               6  phase * condition  MIST3    CFT  Control   
HRV_RMSSD             AT       0              phase      -  MIST1    MIST2   
                               1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
                      FB       1              phase      -  MIST1    MIST3   
HRV_pNN50             AT       0              phase      -  MIST1    MIST2   
                               1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
Time_BL_Glo_HR        AT       0              phase      -  MIST1    MIST2   
                               1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
                               4  phase * condition  MIST1    CFT  Control   
                      BL       1              phase      -  MIST1    MIST3   
                               5  phase * condition  MIST2    CFT  Control   
                               6  phase * condition  MIST3    CFT  Control   
                      FB       0              phase      -  MIST1    MIST2   
                               1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
                               4  phase * condition  MIST1    CFT  Control   
                      RP_CFI   1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
                               4  phase * condition  MIST1    CFT  Control   
                               5  phase * condition  MIST2    CFT  Control   
                               6  phase * condition  MIST3    CFT  Control   
Time_BL_Glo_HRV_RMSSD AT       0              phase      -  MIST1    MIST2   
                               1              phase      -  MIST1    MIST3   
                               2              phase      -  MIST2    MIST3   
                               4  phase * condition  MIST1    CFT  Control   
                               5  phase * condition  MIST2    CFT  Control   
                      FB       1              phase      -  MIST1    MIST3   
                               4  phase * condition  MIST1    CFT  Control   
                      RP_CFI   4  phase * condition  MIST1    CFT  Control   
                               5  phase * condition  MIST2    CFT  Control   
                               6  phase * condition  MIST3    CFT  Control   
Time_BL_Glo_HRV_pNN50 AT       0              phase      -  MIST1

##### Plots

In [32]:
feature = ["HR_Norm"]
subphase = ["BL"]

hr_plot = hr_measures.loc[:, :, :, subphase, feature]
stats_data = stats.results_cat("posthoc").loc[feature, subphase, :]

box_pairs, pvalues = stats.sig_brackets(stats_data, stats_type="interaction", plot_type="multi", x="phase")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=hr_plot.reset_index(), 
    x="phase", 
    y="data", 
    hue="condition", 
    hue_order=hue_order, 
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues},
    legend_loc="upper left",
    ax=ax
)

ax.set_xlabel("Phase")
ax.set_ylabel("$\Delta$ HR during $BL_{Loc}$ [%]")

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

p-value annotation legend:
*: 1.00e-02 < p <= 5.00e-02
**: 1.00e-03 < p <= 1.00e-02
***: p <= 1.00e-03

MIST2_Control v.s. MIST2_CFT: Custom statistical test, P_val:3.240e-02
MIST3_Control v.s. MIST3_CFT: Custom statistical test, P_val:3.000e-03


In [33]:
feature = ["Time_BL_Glo_HR"]
subphase = ["BL"]

hr_plot = hr_measures.loc[:, :, :, subphase, feature]
stats_data = stats.results_cat("posthoc").loc[feature, subphase, :]

box_pairs, pvalues = stats.sig_brackets(stats_data, stats_type="interaction", plot_type="multi", x="phase")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=hr_plot.reset_index(), 
    x="phase", 
    y="data", 
    hue="condition", 
    hue_order=hue_order, 
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues},
    legend_loc="upper left",
    ax=ax
)


ax.set_xlabel("Phase")
ax.set_ylabel("$\hat{t}_{Glo}(HR)$ during $BL_{Loc}$ [%]")

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

p-value annotation legend:
*: 1.00e-02 < p <= 5.00e-02
**: 1.00e-03 < p <= 1.00e-02
***: p <= 1.00e-03

MIST2_Control v.s. MIST2_CFT: Custom statistical test, P_val:3.960e-02
MIST3_Control v.s. MIST3_CFT: Custom statistical test, P_val:6.000e-04
